In [ ]:
#|hide
from compute_benchmarks import *

Found 2 devices
device 0: (null) (null) EGL_EXT_device_drm EGL_EXT_device_drm_render_node
device 1: (null) (null) EGL_MESA_device_software EGL_EXT_device_drm_render_node
eglInitialize: 1.4
egl 1.4, GLES 3.1
Panfrost Mali-G52 (Panfrost) OpenGL ES 3.1 Mesa 22.2.0


## The simplest possible memcpy shader

In [ ]:
#| exports
createTexture(1, 1, output=False)
createTexture(1, 1, texid=1)
showShaderDisassembly("""
#version 310 es
    precision mediump float;

    layout(local_size_x = 1, local_size_y = 1) in;
    layout(rgba32f, binding = 0) uniform mediump readonly image2D img_input;
    layout(rgba32f, binding = 1) uniform mediump writeonly image2D img_output;

    void main() {
      ivec3 start_coords = ivec3(gl_GlobalInvocationID.xyz);
      vec4 x = imageLoad(img_input, start_coords.xy + ivec2(0,0));
      imageStore(img_output, start_coords.yz, x);
    }
""")

FMAs: 0.00% (0 / 4)

clause_0:
ds(0) nbb attr ncph next_attr 
{
    *MKVEC.v2i16 t0, r60, r61
    +LD_ATTR_TEX.f32.v4 t1, t, #0.x, #0.x, @r0
}

clause_1:
ds(0) nbb attr ncph next_store dwb(0) 
{
    *NOP t0
    +MKVEC.v2i16 t1, r61, r62
    *DTSEL_IMM.attribute_1 t0, t1
    +LEA_ATTR_TEX.f32 t1, t, 0x00000000 /* 0.000000 */, 0x00000001 /* 0.000000 */, @r4
}

clause_4:
ds(0) eos store 
{
    *NOP t0
    +ST_CVT.v4 t1, r4, r5, r6, @r0
}

shader0 - MESA_SHADER_COMPUTE shader: 0 inst, 0 bundles, 0 quadwords, 0 registers, 4 threads, 0 loops, 0:0 spills:fills



## Adding some MAC

In [ ]:
createTexture(1, 1, output=False)
createTexture(1, 1, texid=1)
showShaderDisassembly("""
#version 310 es
    precision mediump float;

    layout(local_size_x = 1, local_size_y = 1) in;
    layout(rgba32f, binding = 0) uniform mediump readonly image2D img_input;
    layout(rgba32f, binding = 1) uniform mediump writeonly image2D img_output;

    void main() {
      ivec3 start_coords = ivec3(gl_GlobalInvocationID.xyz);
      vec4 x = imageLoad(img_input, start_coords.xy + ivec2(0,0));
      x = x * x + x;
      imageStore(img_output, start_coords.yz, x);
    }
""")

FMAs: 18.18% (2 / 11)

clause_0:
ds(0) nbb attr ncph next_attr dwb(0) 
{
    *MKVEC.v2i16 t0, r60, r61
    +LD_ATTR_TEX.f32.v4 t1, t, #0.x, #0.x, @r0
}

clause_1:
ds(0) nbb attr ncph next_store dwb(0) 
{
    *NOP t0
    +V2F32_TO_V2F16 r0:t1, r0, r1
    *NOP t0
    +V2F32_TO_V2F16 r1:t1, r2, r3
    *FMA.v2f16 t0, r0, r0, r0
    +F16_TO_F32 r2:t1, t
    *FMA.v2f16 r1:t0, r1, r1, r1
    +F16_TO_F32 r0:t1, t0.h1
    *F16_TO_F32 r3:t0, t0
    +MOV.i32 r4:t1, r2
    *F16_TO_F32 t0, r1.h1
    +MOV.i32 r5:t1, r0
    *MOV.i32 r7:t0, t0
    +MKVEC.v2i16 t1, r61, r62
    *DTSEL_IMM.attribute_1 t0, t1
    +LEA_ATTR_TEX.f32 t1, t, 0x00000000 /* 0.000000 */, 0x00000001 /* 0.000000 */, @r0
}

clause_7:
ds(0) eos store 
{
    *NOP t0
    +MOV.i32 r6:t1, r3
    *NOP t0
    +ST_CVT.v4 t1, r0, r1, r2, @r4
}

shader1 - MESA_SHADER_COMPUTE shader: 0 inst, 0 bundles, 0 quadwords, 0 registers, 4 threads, 0 loops, 0:0 spills:fills



Funny thing, since our local variable `x` is `mediump` (fp16) and our input and output images are fp32 we get lots of F32 to F16 (and back) conversions, not great.

## Fix the datatypes

In [ ]:
createTexture(1, 1, output=False)
createTexture(1, 1, texid=1)
showShaderDisassembly("""
#version 310 es
    precision mediump float;

    layout(local_size_x = 1, local_size_y = 1) in;
    layout(rgba32f, binding = 0) uniform mediump readonly image2D img_input;
    layout(rgba32f, binding = 1) uniform mediump writeonly image2D img_output;

    void main() {
      ivec3 start_coords = ivec3(gl_GlobalInvocationID.xyz);
      highp vec4 x = imageLoad(img_input, start_coords.xy + ivec2(0,0));
      x = x * x + x;
      imageStore(img_output, start_coords.yz, x);
    }
""")

FMAs: 50.00% (4 / 8)

clause_0:
ds(0) nbb attr ncph next_attr 
{
    *MKVEC.v2i16 t0, r60, r61
    +LD_ATTR_TEX.f32.v4 t1, t, #0.x, #0.x, @r0
}

clause_1:
ds(0) nbb attr ncph next_store dwb(0) 
{
    *NOP t0
    +MKVEC.v2i16 t1, r61, r62
    *DTSEL_IMM.attribute_1 t0, t1
    +LEA_ATTR_TEX.f32 t1, t, 0x00000000 /* 0.000000 */, 0x00000001 /* 0.000000 */, @r4
}

clause_4:
ds(0) eos store 
{
    *FMA.f32 r0:t0, r0, r0, r0
    +NOP t1
    *FMA.f32 r1:t0, r1, r1, r1
    +NOP t1
    *FMA.f32 r2:t0, r2, r2, r2
    +NOP t1
    *FMA.f32 r3:t0, r3, r3, r3
    +NOP t1
    *NOP t0
    +ST_CVT.v4 t1, r4, r5, r6, @r0
}

shader2 - MESA_SHADER_COMPUTE shader: 0 inst, 0 bundles, 0 quadwords, 0 registers, 4 threads, 0 loops, 0:0 spills:fills



That's a lot better.

## Add some more multiplications

In [ ]:
createTexture(3, 1, texid=0, output=False)
createTexture(1, 1, texid=1)
showShaderDisassembly("""
    #version 310 es
    precision mediump float;

    layout(local_size_x = 1, local_size_y = 1) in;
    layout(rgba32f, binding = 0) uniform mediump readonly image2D img_input;
    layout(rgba32f, binding = 1) uniform mediump writeonly image2D img_output;

    void main() {
      ivec2 start_coords = ivec2(gl_GlobalInvocationID.xy) * ivec2(2,1);
      highp vec4 x, y;
      x = imageLoad(img_input, start_coords + ivec2(0,0));
      y = imageLoad(img_input, start_coords + ivec2(1,0));
      highp vec4 r = vec4(
          dot(x.xyzw, y.xyzw),
          dot(x.xyzw, y.yxwz),
          dot(x.xyzw, y.zwxy),
          dot(x.xyzw, y.wzyx)
      );
      imageStore(img_output, ivec2(gl_GlobalInvocationID.xy), r);
    }
""")

FMAs: 64.00% (16 / 25)

clause_0:
ds(0) nbb attr ncph next_attr dwb(0) 
{
    *LSHIFT_OR.i32 r0:t0, r60, #0, 0x00000001 /* 0.000000 */
    +NOP t1
    *MKVEC.v2i16 t0, t0, r61
    +LD_ATTR_TEX.f32.v4 t1, t, #0.x, #0.x, @r1
}

clause_3:
ds(0) nbb attr ncph dwb(0) 
{
    *NOP t0
    +IADD.s32 t1, r0, 0x00000001 /* 0.000000 */
    *MKVEC.v2i16 t0, t1, r61
    +LD_ATTR_TEX.f32.v4 t1, t, #0.x, #0.x, @r5
}

clause_6:
ds(0) nbb ncph 
{
    *FMA.f32 t0, r4, r8, #0.neg
    +NOP t1
    *FMA.f32 r0:t0, r3, r7, t0
    +NOP t1
}

clause_8:
ds(0) nbb ncph next_attr 
{
    *FMA.f32 t0, r2, r6, r0
    +NOP t1
    *FMA.f32 r0:t0, r1, r5, t0
    +NOP t1
    *FMA.f32 t0, r4, r7, #0.neg
    +NOP t1
    *FMA.f32 t0, r3, r8, t0
    +NOP t1
    *FMA.f32 t0, r2, r5, t0
    +NOP t1
    *FMA.f32 r9:t0, r1, r6, t0
    +NOP t1
    *FMA.f32 t0, r4, r6, #0.neg
    +NOP t1
    *FMA.f32 r10:t0, r3, r5, t0
    +NOP t1
}

clause_14:
ds(0) nbb attr ncph next_store dwb(0) 
{
    *FMA.f32 t0, r2, r8, r10
    +NOP t1
    *